In [2]:
from matplotlib import pylab
import nengo
import numpy as np

%matplotlib inline

In [224]:
model = nengo.Network(seed=3)
SIM_TIME = 50
SWITCH_TIME = 500
SWITCH_TIME2 = 50
SIM_TIME2 = SWITCH_TIME2*0.001
def fn(j, vals, vals2):
    s = SWITCH_TIME
    vs = vals
    t = 0
    cur = 0
    i = 0
    while True:
    #initial training
        if t >= SIM_TIME:
            break
        j.append(vs[i])
        t = yield [vs[i], 0, 1]
        if t-cur >= 0.001*(s-1):
            cur = t
            i += 1
    s = SWITCH_TIME2
    vs = vals2
    i = 0
    cur = t
    while True:
    #assessment 1
        if i >= len(vals2):
            print 'too far'
            break
        j.append(vs[i])
        t = yield [vs[i], 0, 0]
        if t-cur >= 0.001*(s-1):
            cur = t
            i += 1
    while True:
        t = yield [0, 0, 0]

In [225]:
dist = nengo.dists.Uniform(-1.0, 1.0)
vals = dist.sample((1000*SIM_TIME/SWITCH_TIME)+1)
vals2 = np.linspace(-1, 1, 45)
j=[]
func = fn(j, vals, vals2)
func.next()
func = func.send

In [226]:
with model:
    stim = nengo.Node(func)
    
    sensory = nengo.Ensemble(n_neurons=200, dimensions=3,
                            intercepts=nengo.dists.Uniform(0.87,0.91))
    
    
    spatial = nengo.Ensemble(n_neurons=200, dimensions=2,
                            intercepts=nengo.dists.Uniform(0.87,0.91))
    
    nengo.Connection(stim, sensory[:],
                     function=lambda x: (np.sin(x[0]*np.pi/2), 
                                         np.cos(x[0]*np.pi/2), 
                                         x[1]))
    
    #nengo.Connection(sensory[:2], sensory[2:], synapse=0.2)
    
    c = nengo.Connection(sensory, spatial, function=lambda x: [0,0],
                         learning_rule_type=nengo.PES(learning_rate=3e-4))
        
    error = nengo.Ensemble(n_neurons=100, dimensions=3)
    
    nengo.Connection(stim[2], error[2])
    nengo.Connection(sensory[:2], error[:2],
                     transform=-1)
    nengo.Connection(error, c.learning_rule, 
                     function=lambda x: x[:2] if x[2] > 0.5 else [0, 0])
    #nengo.Connection(spatial, c.learning_rule, synapse=0.1)
    
    nengo.Connection(spatial, error[:2])
    #correct = nengo.Node([0,0])
    #nengo.Connection(correct, error, transform=-1)
    
    error_probe = nengo.Probe(error)
    spatial_probe = nengo.Probe(spatial.neurons)
    spatial_probe_v = nengo.Probe(spatial)
    


In [227]:
sim = nengo.Simulator(model)
sim.run(SIM_TIME)

Simulation finished in 0:00:31.                                                 


In [228]:
def sim_gen(time, cnt):
    cur = 0
    while cur < cnt:
        yield np.mean(sim.data[spatial_probe][-SWITCH_TIME2:], axis=0)
        print sim.data[spatial_probe].shape
        sim.run(time)
        cur += 1
        
def calc_norm(rates):
    #assume that cells with similar peak centers would fall under the same
    #multicellular recording therefore peaks are done relative to all values with similar peaks
    nrm_rates = []
    rate_peaks = np.max(rates, axis=1)
    max_i = np.argmax(rates, axis=0)
    for j, i in enumerate(max_i):
        nrm_rates.append(rates[:, j]/rate_peaks[i])
    nrm_rates = np.array(nrm_rates).transpose()
    return nrm_rates

def align_cut(nrm, dist, num_checks):
    #below uses the protocol from the paper where the center value of the values with >= 50% of
    #the max value is used as the max
    #cut assumes that it is not multimodal
    cut = [[(i, neuron_val) for i, neuron_val in enumerate(nrm[:, col]) 
            if neuron_val >= 0.5] for col in range(nrm.shape[1])]
    maxes = [(i, neuron_vals[len(neuron_vals)/2][0]) 
             for i, neuron_vals in enumerate(cut) 
             if len(neuron_vals) > 0]
    max_cuts = [nrm[mx-dist:mx+dist+1, i] 
                for i, mx in maxes 
                if mx >= dist and mx <= num_checks-(dist+1)]
    return np.array(max_cuts)

def prep_graph(time, cnt, dist):
    sim_g = sim_gen(time, cnt)
    sim_g.next()
    return align_cut(calc_norm(np.array([mean for mean in sim_g])), dist)

def graph_neurons(vals, step):
    for v in range(vals.shape[0]):
        pylab.plot(range(-DIST*step, (DIST+1)*step, step), vals[v, :])

def graph_mean(mean, step):
    pylab.plot(range(-DIST*step, (DIST+1)*step, step), mean)

In [229]:
sim_g = sim_gen(SIM_TIME2, len(vals2))
a=sim_g.next()
print a.shape
mean = np.array([mean for mean in sim_g])

(200,)
(50000, 200)
Simulation finished in 0:00:01.                                                 
(50050, 200)
Simulation finished in 0:00:01.                                                 
(50100, 200)
Simulation finished in 0:00:01.                                                 
(50150, 200)
Simulation finished in 0:00:01.                                                 
(50200, 200)
Simulation finished in 0:00:01.                                                 
(50250, 200)
Simulation finished in 0:00:01.                                                 
(50300, 200)
Simulation finished in 0:00:01.                                                 
(50350, 200)
Simulation finished in 0:00:01.                                                 
(50400, 200)
Simulation finished in 0:00:01.                                                 
(50450, 200)
Simulation finished in 0:00:01.                                                 
(50500, 200)
Simulation finished in 0:00:01.         

In [230]:
n = calc_norm(mean)

/usr/lib64/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in divide


In [231]:
DIST = 10
out = align_cut(n, DIST, len(vals2))

In [232]:
print mean.shape
print n.shape
print out.shape
print [len(x) for x in out]
print np.mean(mean[:, :], axis=1)

(44, 200)
(44, 200)
(45,)
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 21, 21, 21, 21, 21, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21]
[  0.    2.   17.   31.3  22.6  12.    9.2  13.4  12.7  13.6  10.9   7.
  27.7  33.9  10.6  15.2  36.6  48.4  60.6  38.4  30.    3.7  15.1  12.6
   3.2   3.3  29.1  37.3  28.3  16.8  10.1  12.6  11.3  25.5  47.8  34.4
  16.   16.7  28.4  20.4  30.5  27.8  25.8  22.3]


In [234]:
#print j[-(50*45+1):-(50*43)]
print j[1502:1505]
print j[1002]
print j[1503]
print len(j)
#print [(j[i*500+2],np.mean(j[i*500+2:(i+1)*500+2])) for i in range(100)]
print [j[:500*100+1][i*500+3] for i in range(100)]
d = {}
for a in set(j[:50001]):
    d[a] = 0
for a in j[:50001]:
    d[a] += 1
for k in d:
    if d[k] != 500:
        print k, d[k]
print j[:50001][-1]
print j[0]

[0.71236197492622977, 0.71236197492622977, 0.71236197492622977]
-0.237875359683
0.712361974926
52251
[-0.33712848776146598, 0.34189185337210581, -0.23787535968306628, 0.71236197492622977, -0.67184923623671966, -0.99623902934628483, -0.19089636377871355, 0.22225663249388927, -0.43185899405533235, -0.19963889422006309, 0.72703120435681412, 0.86809876074649361, -0.4324228862876407, -0.84155977820803773, -0.74357352294855517, -0.28669169789954929, -0.5571239433288917, -0.69944589409294067, 0.9444337486349943, 0.57760630185501771, 0.60996459350334997, 0.37345430846058969, -0.73470639741538868, 0.082559886701077501, -0.63257732318345061, -0.42616903318514243, 0.19385811730140512, 0.46092814635418455, 0.24594352429433308, -0.72688104320700653, -0.37218109982644609, -0.52921557012783826, 0.83758361202627407, -0.056735157931992575, 0.51682563891840672, 0.63974274622605964, -0.47806422674195503, 0.52296916252519243, 0.56343562061417529, -0.69599931020541339, -0.61941355940636944, -0.171216354903

In [15]:
graph_neurons(vals, STEP)

(50, 200)


In [ ]:
graph_mean(mean_vals, STEP)